<a href="https://colab.research.google.com/github/m0tchy/camera-geometry-tutorial/blob/main/06_perspective.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

この演習では、カメラの投影などを Matplotlib で表示するために、少し工夫をする。
Matplotlib の 3D 表示では、 Z 軸が上向きの右手系であり、これを変更できない。
そこで、プロットする際は、 X, Y, Z 座標を、 X, Z, Y として入れ替えてプロットすることにし、
表示上は Z 軸の正負を入れ替えて、下向きの Y 軸として考えることにする。

以下のコードは、そのような座標系であることを R, G, B 3色の線で表したものを表示して、
そのあとのプロットの準備を行う関数 `init_3d_plot` を定義している。
詳細を理解することはこの演習の本質ではないので、興味があれば読めばよい。

グラフを表示するときはこれを呼び出して、続くプロットコマンドでは Y, Z を逆にすることにする。

この関数は、 $z = 1$ の平面を正方形で表示する。
Z 上の位置は引数で変更できる。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def init_3d_plot(screen_distance=1):

    # 投影面の定義
    screen_vertices = np.array([
        [-1, -1, screen_distance], [1, -1, screen_distance],
        [1, 1, screen_distance], [-1, 1, screen_distance]])
    screen_edges = [0, 1, 2, 3, 0]

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(projection="3d")
    ax.set_xlim(-5, 5)
    ax.set_ylim(-1, 9)  # Z 軸とみなす
    ax.set_zlim(5, -5)  # Y 軸とみなす
    ax.set_xlabel("X")
    ax.set_ylabel("Z")
    ax.set_zlabel("Y")
    ax.view_init(20, -25)

    # X, Z, Y の順で指定する
    ax.plot([0, 0], [-1, 9], [0, 0], ":", color="SILVER")
    ax.plot([0, 1], [0, 0], [0, 0], color="RED")    # X 軸
    ax.plot([0, 0], [0, 0], [0, 1], color="GREEN")  # Y 軸
    ax.plot([0, 0], [0, 1], [0, 0], color="BLUE")   # Z 軸

    ax.plot(screen_vertices[screen_edges, 0], screen_vertices[screen_edges, 2], screen_vertices[screen_edges, 1], color="GRAY")

    return fig, ax

# plt.figure などの代わりにこの関数を利用する
fig, ax = init_3d_plot()
fig, ax = init_3d_plot(3)


# 復習：空間の点

3次元空間の点は、 4 次元ベクトルの同次座標で表現できる。
実際の座標は、第4成分で割った第1,2,3成分である。

Numpy 配列はインデックスが 0 から始まるので、成分の番号とのずれに注意。

In [ ]:
# 空間の点（同次座標） 複数の点を各行に並べる
X_ = np.array([
    [0, 0, 4, 1], 
    [3, -3, 8, 1], 
    [-2, -2, 8, 1]
])

fig, ax = init_3d_plot()

# 普通の座標に変換
XX = X_[:, 0] / X_[:, 3]
XY = X_[:, 1] / X_[:, 3]
XZ = X_[:, 2] / X_[:, 3]
ax.scatter(XX, XZ, XY, color="BLACK")  # 表示のために Y, Z を入れ替える

# 透視投影

ピンホールカメラモデルと呼ばれる普通のカメラの基本的な仕組みは、透視投影によって空間の点を平面に写す。

## 平面 Z = 1 への投影

この資料では、3次元の空間座標は主に大文字で表す。

点 $\boldsymbol{X} = [X, Y, Z]^\top$ を原点に向かって投影面へ投影した点を $\boldsymbol{X}' = [X', Y', Z']$ とする。
この点の対応を**透視投影** (perspective projection, perspective transformation) という。
このとき原点を**光学中心** (optical center)・カメラ中心 (camera center) ・**投影中心** (projection center)などと呼び、
投影面への距離を**焦点距離** (focal length) と呼ぶ。
また、原点から投影面へ垂直に伸ばした直線を**光軸** (optical axis, principal axis)、
その交点を**主点** (principal point) とか**画像中心** (image center) という。  

投影元の点と原点とをつなぐ直線を光線 (ray) と呼ぶことにすれば、投影点は投影面と光線の交点である。

投影面までの距離を $1$ とすると、その Z 座標が $1$ であるということである。
光線上の点は、 X, Y, Z 方向に等倍した点なので、 Z 座標が $1$ になるように全部を割ればよいことがわかる。

$$
    \begin{align}
        X' &= X / Z \\
        Y' &= Y / Z \\
        Z' &= Z / Z = 1
    \end{align}
$$
これは、線形変換ではないことがわかる（なぜなら線形変換は $X,Y,Z$ の線形和、つまり何倍かして足し合わせることだけしかできないから。）。
当然、アフィン変換でもない。
したがって、行列の掛け算によって計算することができないように思える。

しかし、同次座標で考えるとこの割り算をうまく扱うことができる。
全体を定数倍したものは、元の点と同じ座標であると決めたのが同次座標である。
したがって、この投影された点の同次座標を
$$
    \begin{align}
    \begin{bmatrix} X/Z \\ Y/Z \\ Z/Z \\ 1 \end{bmatrix}
    \sim 
    \begin{bmatrix} X \\ Y \\ Z \\ Z \end{bmatrix}
    \end{align}
$$
と変形して、 $[X, Y, Z, Z]^\top$ という座標が求まればよいことになる。

この変換を行列で表すと以下のようになる。
$\boldsymbol{X}$ の同次座標 $\tilde{\boldsymbol{X}} = [X, Y, Z, 1]^\top$
に対して、
投影された点が $\tilde{\boldsymbol{X}'} = [X/Z, Y/Z, 1, 1]^\top \sim [X, Y, Z, Z]^\top$ となるような変換行列 $P$ を考えて、
$$
    \tilde{\boldsymbol{X}'} \sim P\tilde{\boldsymbol{X}}
$$
と計算すればよい。
この行列は以下のように書くことができる。
$$
    P = \begin{bmatrix}
         1 & 0 & 0 & 0 \\
         0 & 1 & 0 & 0 \\
         0 & 0 & 1 & 0 \\
         0 & 0 & 1 & 0
    \end{bmatrix}
$$
この行列は最終行が $[0, 0, 0, 1]$ でないのでアフィン変換ではない。
この行列が表す変換は**射影変換** (projective transform) と呼ばれるものの一種であり、
特に**透視投影変換** (perspective transform) と呼ばれる。

以上をまとめると、このようになる。
$$
    \tilde{\boldsymbol{X}'}
    = \begin{bmatrix} X' \\ Y' \\ Z' \\ 1 \end{bmatrix}
    = \begin{bmatrix} X/Z \\ Y/Z \\ 1 \\ 1 \end{bmatrix}
    \sim \begin{bmatrix} X \\ Y \\ Z \\ Z \end{bmatrix}
    = \begin{bmatrix}
         1 & 0 & 0 & 0 \\
         0 & 1 & 0 & 0 \\
         0 & 0 & 1 & 0 \\
         0 & 0 & 1 & 0
    \end{bmatrix}
    \begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix}
    = P\tilde{\boldsymbol{X}}
$$



In [ ]:
projection = np.array([
    [1, 0, 0, 0], 
    [0, 1, 0, 0], 
    [0, 0, 1, 0], 
    [0, 0, 1, 0]])
# projection = np.eye(4, 4)

# projected onto the screen
X2_ = (projection @ X_.T).T
print(X2_)

fig, ax = init_3d_plot()

# 光線を書く
ax.quiver(0, 0, 0, XX, XZ, XY, color="SILVER", linestyle="dashed", arrow_length_ratio=0)

# 普通の座標に変換
X2X = X2_[:, 0] / X2_[:, 3]
X2Y = X2_[:, 1] / X2_[:, 3]
X2Z = X2_[:, 2] / X2_[:, 3]

# 点を打つ
ax.scatter(XX, XZ, XY, color="BLACK")
ax.scatter(X2X, X2Z, X2Y, color="BLACK")

## 平面 Z = f への投影

より一般に、
投影面までの距離を $f$ とすると、その Z 座標が $f$ になればよいので、
$$
    \begin{align}
        X' &= (X / Z)f \\
        Y' &= (Y / Z)f \\
        Z' &= (Z / Z)f = f
    \end{align}
$$
となり、その同次座標は、
$$
    \begin{align}
    \begin{bmatrix} fX/Z \\ fY/Z \\ f \\ 1 \end{bmatrix}
    \sim 
    \begin{bmatrix} fX \\ fY \\ fZ \\ Z \end{bmatrix}
    \end{align}
$$
である。したがって、透視投影変換行列 $P_f$ を
$$
    P_f = \begin{bmatrix}
         f & 0 & 0 & 0 \\
         0 & f & 0 & 0 \\
         0 & 0 & f & 0 \\
         0 & 0 & 1 & 0
     \end{bmatrix}
$$
とすれば、 $Z = f$ 上への投影は、
$$
\tilde{\boldsymbol{X}'} \sim P_f \tilde{\boldsymbol{X}}
$$
となる。

以上をまとめると、このようになる。
$$
    \tilde{\boldsymbol{X}'}
    = \begin{bmatrix} X' \\ Y' \\ Z' \\ 1 \end{bmatrix}
    = \begin{bmatrix} fX/Z \\ fY/Z \\ f \\ 1 \end{bmatrix}
    \sim \begin{bmatrix} fX \\ fY \\ fZ \\ Z \end{bmatrix}
    = \begin{bmatrix}
         f & 0 & 0 & 0 \\
         0 & f & 0 & 0 \\
         0 & 0 & f & 0 \\
         0 & 0 & 1 & 0
    \end{bmatrix}
    \begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix}
    = P_f\tilde{\boldsymbol{X}}
$$



In [ ]:
# distance to the screen from the origin
f = 2

projection = np.array([
    [f, 0, 0, 0], 
    [0, f, 0, 0], 
    [0, 0, f, 0], 
    [0, 0, 1, 0]])

# projected onto the screen
X2_ = (projection @ X_.T).T
print(X2_)

fig, ax = init_3d_plot(f)  # 引数でスクリーンまでの距離を指定する

# 光線を書く
ax.quiver(0, 0, 0, XX, XZ, XY, color="SILVER", linestyle="dashed", arrow_length_ratio=0)

# 普通の座標に変換
X2X = X2_[:, 0] / X2_[:, 3]
X2Y = X2_[:, 1] / X2_[:, 3]
X2Z = X2_[:, 2] / X2_[:, 3]

# 点を打つ
ax.scatter(XX, XZ, XY, color="BLACK")
ax.scatter(X2X, X2Z, X2Y, color="BLACK")

## 練習問題

前回の演習のように、点の数を増やし、各点をつなぐ辺を適当に定義して、空間上と投影平面上それぞれに表示しなさい。


## 解答例
最初は見ないでやること。

In [ ]:
# 全体的には上のコードと同じ
# X_, X2_, projection は定義されているとする

edges = [0, 1, 2, 0]  # このようなものを追加

fig, ax = init_3d_plot(f)  # 引数でスクリーンまでの距離を指定する

# 普通の座標に変換
X2X = X2_[:, 0] / X2_[:, 3]
X2Y = X2_[:, 1] / X2_[:, 3]
X2Z = X2_[:, 2] / X2_[:, 3]

# 点を打つ
ax.scatter(XX, XZ, XY, color="BLACK")
ax.scatter(X2X, X2Z, X2Y, color="BLACK")

# 辺の描画
ax.plot(XX[edges], XZ[edges], XY[edges])
ax.plot(X2X[edges], X2Z[edges], X2Y[edges])

# 投影面上の図形の描画 (次回の予習)



投影面上に描かれる図形について考えてみよう。



$f = 1$ の場合、つまり投影された点の Z 座標が $1$ の場合、
非同次座標は以下の式だった。

$$
    \boldsymbol{X}'
    = \begin{bmatrix} X' \\ Y' \\ Z' \end{bmatrix}
    = \begin{bmatrix} X/Z \\ Y/Z \\ 1 \end{bmatrix}
$$

これは**投影平面上の2次元座標 $(X/Z, Y/Z)$ の同次座標**であるとみなすことができる。
また、一般の $f$  の場合、
$$
    \begin{bmatrix} X' \\ Y' \\ Z' \end{bmatrix}
    = \begin{bmatrix} fX/Z \\ fY/Z \\ f \end{bmatrix}
$$
なので、**投影平面上の2次元座標**としては $(fX/Z, fY/Z)$ であり、
その同次座標は $(fX/Z, fY/Z, 1) \sim (fX, fY, Z)$ である。

$$
    \tilde{\boldsymbol{x}}'_f
    = \begin{bmatrix} fX/Z \\ fY/Z \\ 1 \end{bmatrix}
    \sim \begin{bmatrix} fX \\ fY \\ Z \end{bmatrix}
$$

この座標は、上で説明した $P_f$ で変換された座標 $(fX, fY, fZ, Z)$ の**第3成分**を抜いたものなので、
この行列を再定義して、3次元の同次座標を2次元の同次座標に直接変換するように
$$
    \tilde{\boldsymbol{x}}'_f 
    \sim P_f \tilde{\boldsymbol{X}'} =
    \begin{bmatrix} f & 0 & 0 & 0 \\ 0 & f & 0 & 0 \\ 0 & 0 & 1 & 0 \end{bmatrix} \tilde{\boldsymbol{X}'}
$$
と書く。

$f$ の違いは、主点を中心とした拡大縮小であることが幾何的な関係から分かる。
この性質はこの行列を以下のように分解できることで理解できる。
$$
    P_f =
    \begin{bmatrix} f & 0 & 0 \\ 0 & f & 0 \\ 0 & 0 & 1 \end{bmatrix}
    \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \end{bmatrix} = K \left[ I_{3 \times 3} \mid \mathbf{0}_3 \right]
$$
これは、（左から順に変換が行われるので）まず第3成分を取り除き、そのあと拡大縮小することを表現している。

左のアフィン変換 $K$ はカメラ固有の性質を表しているものなので、カメラ校正行列 (camera calibration matrix) とか、内部パラメター行列 (intrinsic parameters matrix) などと呼ばれる。